<center>
    <h1>Recherche Opérationnelle : TP1 et 2</h1>
    <h2>Hamza Mouddene</h2>
    <h2>Anass Tyoubi</h2>
</center>

## 1 Exemple de Modélisation en PL(NE)
***

Un nouveau client se propose d’acquérir la production de ciment de l’usine ALPHA à des prix plus avantageux que le marché, notamment 55 euros pour le type 1 et 100 pour le type 2, à condition de garantir une livraison quotidienne d’au moins 10 tonnes dont au moins 5 tonnes de ciment de type 1.

#### Modélisation et résolution du problème

In [1]:
#importer les packages utiles, le manager de package Pkg etant un package
import Pkg; Pkg.add("Cbc")
Pkg.add("JuMP")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100,0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
N = 2 # nombre de ciments disponibles
c = [50,70] # prix de vente par ciment
b = [360,480] # temps de disponibilité du four et de l'atelier
A = [40 12;20 30] # temps nécessaire de calcination (four) et de broyage (atelier) par ciment

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, ciment[1:N] >= 0)

# define objective function
@objective(model, Max, sum(c[i]*ciment[i] for i in 1:N))

# define constraints
for i in 1:length(b)
    @constraint(model, sum(A[i,j]*ciment[j] for j in 1:N) <= b[i])
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 2 (0) rows, 2 (0) columns and 4 (0) elements
0  Obj -0 Dual inf 120 (2)
0  Obj -0 Dual inf 120 (2)
2  Obj 1137.5
Optimal - objective value 1137.5
Optimal objective 1137.5 - 2 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.11



#### Tests avec l’instance disponible sur le sujet

In [3]:
# print solution
println("Solution obtenue:")
println("\t benefice = $(objective_value(model))")
for i in 1:N
    println("\t quantite de ciment $i = $(value(ciment[i]))")
end

Solution obtenue:
	 benefice = 1137.5
	 quantite de ciment 1 = 5.249999999999999
	 quantite de ciment 2 = 12.5


## 2 Applications en optimisation pour l’e-commerce
***

Parmi les problématiques d’optimisation émergeant en e-commerce, se trouvent l’affectation de commandes de clients aux magasins, compte-tenu des coûts associés à la livraison des colis, à la préparation des commandes et à la gestion des différents stocks. Nous nous intéresserons particulièrement au problème d’affectation de commandes et tournées de véhicules pour différents magasins d’une même enseigne ou franchise. Nous nous plaçons
donc dans la peau du gestionnaire de l’ensemble des magasins, qui doit d´epenser le moins
d’argent possible pour satisfaire les demandes des clients.

### 2.1 Cas particulier 1
***

Les tableaux (a), (b) et (c) représentent à titre d’exemple des demandes en fluide émanant
de différentes commandes et les coûts pour qu’une unité de fluide soit disponible dans un
magasin. Chaque magasin dispose d’un volume de stockage limité.

| A  | F1 | F2 |
|----|----|----|
| D1 | 2  | 0  |
| D2 | 1  | 3  |

| B  | F1  | F2 |
|----|-----|----|
| M1 | 2.5 | 1  |
| M2 | 1   | 2  |
| M3 | 2   | 1  |


| C  | F1 | F2 |
|----|----|----|
| M1 | 1  | 1  |
| M2 | 2  | 3  |
| M3 | 3  | 2  |

#### Modélisation et résolution du problème

In [4]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
n_fluides = 2               # Nombre de fluides disponibles
n_demandes = 2              # Nombre de demandes 
n_magasins = 3              # Nombre de magasins
demandes = [2 0; 1 3]       # Les demandes figurantes dans le tableau (a)
stocks = [2.5 1; 1 2; 2 1]  # Le stock par magasin figurant dans le tableau (b)
couts = [1 1; 2 3; 3 2]     # Le cout par fluide pour les différents magasins figurant dans le tableau (c)

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, quantites[1 : n_magasins, 1 : n_fluides, 1 : n_demandes] >= 0)

# define objective function
A = sum(quantites[:, :, k] for k in 1 : n_demandes)
B = A .* couts
@objective(model, Min, sum(B))

# define constraints
for i in 1 : n_fluides 
    @constraint(model, sum(A[:, i]) >= sum(demandes[:, i]))
end

for i in 1 : n_magasins
    for j in 1 : n_fluides
        @constraint(model, A[i, j] <= stocks[i, j])
    end
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 2 (-6) rows, 6 (-6) columns and 6 (-18) elements
0  Obj 2.6999999 Primal inf 5.099998 (2)
2  Obj 9.5
Optimal - objective value 9.5
After Postsolve, objective 9.5, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 9.5 - 2 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



#### Tests avec l’instance disponible sur le sujet

In [5]:
# print solution
println("Solution obtenue:")
println("\t benefice = $(objective_value(model))\n")
for i in 1 : n_magasins
    for j in 1 : n_fluides
        for k in 1 : n_demandes
            println("\t Pour la demande $k : Il faut prendre $(value(quantites[i, j, k])) unité pour le fluide $j du magasin $i")
        end
    end
end

Solution obtenue:
	 benefice = 9.5

	 Pour la demande 1 : Il faut prendre 2.5 unité pour le fluide 1 du magasin 1
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 1
	 Pour la demande 1 : Il faut prendre 1.0 unité pour le fluide 2 du magasin 1
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 2 du magasin 1
	 Pour la demande 1 : Il faut prendre 0.5 unité pour le fluide 1 du magasin 2
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 2
	 Pour la demande 1 : Il faut prendre 1.0 unité pour le fluide 2 du magasin 2
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 2 du magasin 2
	 Pour la demande 1 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 3
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 3
	 Pour la demande 1 : Il faut prendre 1.0 unité pour le fluide 2 du magasin 3
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 2 du magasin 3


### 2.2 Cas particulier 2
***

En réalité, il ne s’agit pas de fluide mais de produits préconditionnés et une commande
d’un client peut être constituée de plusieurs produits en quantités différentes. Le problème
d’affectation consiste à déterminer le nombre de produits de chaque type livrés par chaque
magasin à chaque client. Modifier et résoudre la formulation précédente pour tenir compte
de la discrétisation de la demande. Tester avec différents jeux de données.

#### Modélisation et résolution du problème

In [6]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
n_fluides = 2               # Nombre de fluides disponibles
n_demandes = 2              # Nombre de demandes 
n_magasins = 3              # Nombre de magasins
demandes = [2 0; 1 3]       # Les demandes figurantes dans le tableau (a)
stocks = [2.5 1; 1 2; 2 1]  # Le stock par magasin figurant dans le tableau (b)
couts = [1 1; 2 3; 3 2]     # Le cout par fluide pour les différents magasins figurant dans le tableau (c)

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, quantites[1 : n_magasins, 1 : n_fluides, 1 : n_demandes] >= 0, Int)

# define objective function
A = sum(quantites[:, :, k] for k in 1 : n_demandes)
B = A .* couts
@objective(model, Min, sum(B))

# define constraints
for i in 1 : n_fluides 
    @constraint(model, sum(A[:, i]) >= sum(demandes[:, i]))
end

for i in 1 : n_magasins
    for j in 1 : n_fluides
        @constraint(model, A[i, j] <= stocks[i, j])
    end
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 9.5 - 0.00 seconds
Cgl0004I processed model has 2 rows, 6 columns (6 integer (3 of which binary)) and 6 elements
Cbc0012I Integer solution of 10 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 10, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 10 to 10
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after add

#### Tests avec différents jeux de données

In [7]:
# print solution
println("Solution obtenue:")
println("\t benefice = $(objective_value(model))\n")
for i in 1 : n_magasins
    for j in 1 : n_fluides
        for k in 1 : n_demandes
            println("\t Pour la demande $k : Il faut prendre $(value(quantites[i, j, k])) unité pour le fluide $j du magasin $i")
        end
        println("\n")
    end
end

Solution obtenue:
	 benefice = 10.0

	 Pour la demande 1 : Il faut prendre 2.0 unité pour le fluide 1 du magasin 1
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 1


	 Pour la demande 1 : Il faut prendre 1.0 unité pour le fluide 2 du magasin 1
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 2 du magasin 1


	 Pour la demande 1 : Il faut prendre 1.0 unité pour le fluide 1 du magasin 2
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 2


	 Pour la demande 1 : Il faut prendre 1.0 unité pour le fluide 2 du magasin 2
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 2 du magasin 2


	 Pour la demande 1 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 3
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 3


	 Pour la demande 1 : Il faut prendre 1.0 unité pour le fluide 2 du magasin 3
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 2 du magasin 3




### 2.3 Cas particulier 3
***

A présent on souhaite prendre en compte les coûts d’expédition des colis des magasins aux
clients. Chaque magasin expédie, vers chaque client qu’il dessert, un unique colis contenant
tous les produits fournis par ce magasin à ce client. Modifier la formulation précédente pour
modéliser le problème résultant et résoudre avec les données du tableau (d). Tester ensuite
avec différents jeux de données.

| D  | M1 | M2 | M3 |
|----|----|----|----|
| D1 | 1  | 0  | 0  |
| D2 | 0  | 2  | 1  |

#### Modélisation et résolution du problème

In [8]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
n_fluides = 2                      # Nombre de fluides disponibles
n_demandes = 2                     # Nombre de demandes 
n_magasins = 3                     # Nombre de magasins
demandes = [2 0; 1 3]              # Les demandes figurantes dans le tableau (a)
stocks = [2.5 1; 1 2; 2 1]         # Le stock par magasin figurant dans le tableau (b)
couts = [1 1; 2 3; 3 2]            # Le cout par fluide pour les différents magasins figurant dans le tableau (c)
expedition = [1 0; 0 2; 0 1]       # Le cout d'expédition d'un colis figurant sur le tableau (d)

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, quantites[1 : n_magasins, 1 : n_fluides, 1 : n_demandes] >= 0, Int)
@variable(model, demandes_magasin[1 : n_magasins, 1 : n_demandes], Bin)

# define objective function
A = sum(quantites[:, :, k] for k in 1 : n_demandes)
B = A .* couts + demandes_magasin .* expedition
@objective(model, Min, sum(B))

# define constraints
for i in 1 : n_fluides
    @constraint(model, sum(A[:, i]) == sum(demandes[:, i]))
end

for i in 1 : n_magasins
    for j in 1 : n_fluides
        @constraint(model, (A[i, j] <= stocks[i, j]))
    end
end

for i in 1 : n_magasins
    for j in 1 : n_fluides
        @constraint(model, A[i, j] / sum(demandes[j, :]) <= demandes_magasin[i, j])
        @constraint(model,demandes_magasin[i, j] <= sum(quantites[i, :, j]))
    end
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 11.75 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 10 strengthened rows, 0 substitutions
Cgl0004I processed model has 20 rows, 18 columns (18 integer (12 of which binary)) and 67 elements
Cbc0012I Integer solution of 14 found by DiveCoefficient after 0 iterations and 0 nodes (0.03 seconds)
Cbc0038I Full problem 20 rows 18 columns, reduced to 0 rows 0 columns
Cbc0006I The LP relaxation is infeasible or too expensive
Cbc0013I At root node, 0 cuts changed objective from 13 to 14 in 1 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 3 column cuts (3 active)  in 0.000 seconds - new frequency is 1
Cbc0014I Cut generator 1 (Gomory) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut generator 2 (Knapsack) - 0 row cuts average 0.

#### Tests avec l’instance disponible sur le sujet

In [9]:
# print solution
println("Solution obtenue:\n")
println("\t benefice = $(objective_value(model))\n")
for i in 1 : n_magasins
    for j in 1 : n_fluides
        for k in 1 : n_demandes
            println("\t Pour la demande $k : Il faut prendre $(value(quantites[i, j, k])) unité pour le fluide $j du magasin $i")
        end
        println("\n")
    end
end

Solution obtenue:

	 benefice = 14.0

	 Pour la demande 1 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 1
	 Pour la demande 2 : Il faut prendre 2.0 unité pour le fluide 1 du magasin 1


	 Pour la demande 1 : Il faut prendre 1.0 unité pour le fluide 2 du magasin 1
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 2 du magasin 1


	 Pour la demande 1 : Il faut prendre 1.0 unité pour le fluide 1 du magasin 2
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 2


	 Pour la demande 1 : Il faut prendre 0.0 unité pour le fluide 2 du magasin 2
	 Pour la demande 2 : Il faut prendre 1.0 unité pour le fluide 2 du magasin 2


	 Pour la demande 1 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 3
	 Pour la demande 2 : Il faut prendre 0.0 unité pour le fluide 1 du magasin 3


	 Pour la demande 1 : Il faut prendre 0.0 unité pour le fluide 2 du magasin 3
	 Pour la demande 2 : Il faut prendre 1.0 unité pour le fluide 2 du magasin 3




###  2.4 Cahier des charges du problème général
***

L’objectif est de mod´eliser un problème de minimisation des trajets de livraisons des
commandes (i.e. distance à parcourir) pour les livreurs des magasins. On fera donc abstraction des autres coûts pour se focaliser sur un problềme mono-objectif. Le PLNE proposé
doit prendre en entrée les données suivantes : les commandes à satisfaire (quantités et types
de produits), les niveaux de stocks disponibles au sein des différents magasins, les temps de
trajet entre les différents sites. A partir de ces données, le modèle doit pouvoir d´eterminer

comment répartir les commandes entre les magasins et quelle est la tournée de livraison que
doit réaliser le livreur de chaque magasin.

#### Modélisation et résolution du problème

In [38]:
using LinearAlgebra
include("2_4.jl")

# data
inputfilepath = "data_2-4"
inputfilename = "Data_test_5_2_3.txt"
nb_dem, nb_prod, nb_mag, nb_noeuds, S, Q, R = read_data_24(inputfilepath, inputfilename);

# set Optimizer
model = Model(Cbc.Optimizer)

# define Variables
@variable(model, ID[1:(1+nb_dem),1:(1+nb_dem), 1:nb_mag],Bin)
@variable(model, PM[1:nb_mag, 1:nb_prod, 1:nb_dem] >= 0,Int)


# define Objective function                  
@objective(model, Min, sum(reshape(sum(ID[2:(1+nb_dem),2:(1+nb_dem),:],dims = 3),nb_dem,nb_dem).*reduce(hcat,R[1+nb_mag:nb_dem+nb_mag])[1+nb_mag:nb_dem+nb_mag,:]
                           )+sum((ID[2:1+nb_dem,1,:] + ID[1,2:1+nb_dem,:]).*reduce(hcat,R[1:nb_mag])[1+nb_mag:nb_mag+nb_dem,:]))

# define Constraints
for i in 1:nb_mag
    @constraint(model, sum(ID[1,2:1+nb_dem,i]) <= sum(PM[i,:,:]))
    @constraint(model, sum(PM[i,:,:]) /(1 + sum(S[i][:]))<= sum(ID[1,2:1+nb_dem,i]))
    for j in 1:1+nb_dem
        @constraint(model, sum(ID[j,:,i]) <= sum(PM[i,:,:]))
        @constraint(model, sum(PM[i,:,:]) /(1 + sum(S[i][:]))<= sum(ID[j,:,i]))
        @constraint(model, sum(ID[:,j,i]) <= sum(PM[i,:,:]))
        @constraint(model, sum(PM[i,:,:]) /(1 + sum(S[i][:]))<= sum(ID[:,j,i]))
    end
    for j in 1:nb_dem 
        @constraint(model, sum(ID[j,1:j,i])+sum(ID[1:j,j,i]) <= 1)
    end
    @constraint(model, sum(ID[2:1+nb_dem,1,i]) <= sum(PM[i,:,:]))
    @constraint(model, sum(PM[i,:,:]) /(1 + sum(S[i][:]))<= sum(ID[2:1+nb_dem,1,i]))
    @constraint(model, tr(ID[:,:,i]) == 0)
end

for i in 1:nb_dem
    for j in 1:nb_prod
        @constraint(model, sum(PM[:,j,i]) == Q[i][j])
    end
end

for i in 1:nb_mag
    for j in 1:nb_prod
        @constraint(model,sum(PM[i,j,:]) <= S[i][j])
    end
end

for i in 1:nb_dem
    for j in 1:nb_mag
        @constraint(model,sum(ID[1+i,:,j]) <= sum(PM[j,:,i]))
        @constraint(model,sum(ID[:,1+i,j]) <= sum(PM[j,:,i]))
        @constraint(model, sum(PM[j,:,i]) /( 1 + sum(Q[i][:])) <= sum(ID[1+i,:,j]))
        @constraint(model, sum(PM[j,:,i]) /( 1 + sum(Q[i][:])) <= sum(ID[:,1+i,j]))
        
    end
end

# run Optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3035.17 - 0.00 seconds
Cgl0002I 12 variables fixed
Cgl0004I processed model has 102 rows, 75 columns (75 integer (64 of which binary)) and 1350 elements
Cbc0012I Integer solution of 4133 found by DiveCoefficient after 1558 iterations and 0 nodes (0.28 seconds)
Cbc0031I 27 added rows had average density of 12.444444
Cbc0013I At root node, 27 cuts changed objective from 3035.1667 to 4132.9951 in 64 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.017 seconds - new frequency is -100
Cbc0014I Cut generator 1 (Gomory) - 1367 row cuts average 69.9 elements, 0 column cuts (0 active)  in 0.028 seconds - new frequency is 1
Cbc0014I Cut generator 2 (Knapsack) - 169 row cuts average 6.0 elements, 0 column cuts (0 active)  in 0.020 seconds - new frequency is 1
Cbc0014I Cut

#### Tests avec l’instance disponible sur le fichier data_2-4/Data_test_5_1_2.txt

In [35]:
println("Solution obtenue:")
println("\t Benefice = $(objective_value(model))\n")

Solution obtenue:
	 Benefice = 4133.0



## 3 Bonus
***

Dans cette partie, on va modéliser avec Julia les versions génériques des problèmes vu en TD.

### 3.1 Encore du ciment
***

Un nouveau client se propose d’acquérir la production de ciment de l’usine ALPHA à des prix plus avantageux que le marché, notamment 55 euros pour le type 1 et 100 pour le type 2, à condition de garantir une livraison quotidienne d’au moins 10 tonnes dont au moins 5 tonnes de ciment de type 1.

#### Modélisation d'une forme générique

In [13]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
Nb_ciment = 2 # Nombre de ciment
C = [50; 70] # Coût ciment
Temps = [40 20; 12 30]
Temps_Limite = [360; 480]
Seuil_Min = [5; 0] # Seuil Minimal Ciment
Seuil = 10

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Ciment[1:Nb_ciment] >= 0)

# define objective function
@objective(model, Max, sum(C.*Ciment))

# define constraints Four Limité d'utilisation / jour
for i in 1:Nb_ciment
    @constraint(model, sum(Temps[:,i].*Ciment) <= Temps_Limite[i])
    @constraint(model, Ciment[i] >= Seuil_Min[i])
end

# define constraints Seuil Minimal Ciment
@constraint(model, sum(Ciment) >= Seuil)

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 3 (-2) rows, 2 (0) columns and 6 (-2) elements
0  Obj 342.99801 Primal inf 3.6714561 (1) Dual inf 120 (2)
2  Obj 1137.5
Optimal - objective value 1137.5
After Postsolve, objective 1137.5, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1137.5 - 2 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



#### Tests avec l’instance disponible sur le sujet

In [14]:
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:Nb_ciment
    println("\t Quantite de Ciment $i = $(value(Ciment[i]))")
end

la solution est : 
	 coût = 1137.5
	 Quantite de Ciment 1 = 5.249999999999999
	 Quantite de Ciment 2 = 12.5


### 3.2 Des voitures
***

#### Modélisation d'une forme générique

In [17]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
Nb_Voitures = 2 # Nombre de Voitures
C = [10000; 9000] # Coût
Max_heure = 60
Heure = [0.06; 0.05]
Max_surface = 15000
Surface = [10; 20]
Max_Luxe = 800

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Voitures[1:Nb_Voitures] >= 0, Int)

# define objective function
@objective(model, Max, sum(C.*Voitures))

# define constraints
@constraint(model, sum(Heure.*Voitures) <= Max_heure)
@constraint(model, sum(Surface.*Voitures) <= Max_surface)
@constraint(model, Voitures[1] <= Max_Luxe)

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 1.02857e+07 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (2 integer (0 of which binary)) and 4 elements
Cbc0012I Integer solution of -10272000 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0012I Integer solution of -10274000 found by DiveCoefficient after 5 iterations and 0 nodes (0.01 seconds)
Cbc0031I 2 added rows had average density of 2
Cbc0013I At root node, 2 cuts changed objective from -10285714 to -10284000 in 21 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 49 column cuts (49 active)  in 0.002 seconds - new frequency is 1
Cbc0014I Cut generator 1 (Gomory) - 7 row cuts average 2.0 elements, 0 column cuts (0 active)  in 0.001 seconds - new frequency is 1
Cbc0014I Cut generator 2 (Knapsack) - 0 row cuts average 0.0 elements, 0 column cuts

#### Tests avec l’instance disponible sur le sujet

In [18]:
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:Nb_Voitures
    println("\t Quantite de Voiture $i = $(value(Voitures[i]))")
end

la solution est : 
	 coût = 1.0284e7
	 Quantite de Voiture 1 = 645.0
	 Quantite de Voiture 2 = 426.0


### 3.3 De la teinture
***

La teinturerie ZETA peut utiliser indifféremment deux produits pour colorer du tissu
brut en couleur indigo. Ces produits, appelés I 1 et I 2 , contiennent trois substances A, B et
C dans les concentrations suivantes (exprimées en g par kg de produit) :
I 1
I 2
A
500
400
B
C
150 20
50 0
Dans un bain permettant de teinter 10 kg de tissu, il faut au moins 500 g de la substance
A, 100 g de la substance B et 5 g de la substance C. De plus, la quantité de substance C
ne doit pas dépasser 15 g par bain. Le produit I 1 coûte 10 e par kg, tandis que le produit
I 2 coûte 20 e par kg. ZETA souhaite savoir la somme minimale à débourser pour pouvoir
colorer 10 kg de tissu.

#### Modélisation d'une forme générique

In [20]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
Nb_Teinture = 2 # Nombre de teinture
Nb_seuil = 3
C = [10; 20] # Coût teinture
cout_Seuil_Min = [500 150 20 ;400 50 0]
cout_Seuil_Max = [20;0]
Seuil_Min = [500; 100; 5] # Seuil Minimal
Seuil_Max = 15

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Teinture[1:Nb_Teinture] >= 0)

# define objective function
@objective(model, Min, sum(C.*Teinture))

# define constraints  Seuil Minimal
for i in 1:Nb_seuil
    @constraint(model, sum(cout_Seuil_Min[:,i].*Teinture) >= Seuil_Min[i])
end

# define constraints Seuil Maximal
@constraint(model, sum(cout_Seuil_Max.*Teinture) <= Seuil_Max)

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
0  Obj 6.7499998 Primal inf 0.92583134 (2)
1  Obj 13.75
Optimal - objective value 13.75
After Postsolve, objective 13.75, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 13.75 - 1 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



#### Tests avec l’instance disponible sur le sujet

In [21]:
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:Nb_Teinture
    println("\t Quantite de Ciment $i = $(value(Teinture[i]))")
end

la solution est : 
	 coût = 13.75
	 Quantite de Ciment 1 = 0.75
	 Quantite de Ciment 2 = 0.3125


### 3.4 Des armoires et des tables
***

L’entreprise GAMMA fabrique entre-autre des armoires et des tables :
— Tout article commencé doit être terminé le jour même.
— Chaque article nécessite 1 heure de travail.
— Une armoire nécessite 9 m de bois, alors qu’une table n’en nécessite que 5 m
— GAMMA dispose, chaque jour, de 6 heures de travail et de 45 m2 de bois
— Une armoire rapporte 8 e de bénéfice, alors qu’une table ne rapporte que 5 e.
Comment répartir le travail quotidien entre armoires et tables, de manière à maximiser le
bénéfice ?

#### Modélisation d'une forme générique

In [23]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
Nb_Var = 2 # Nombre de Variables (Table, Armoires)
C = [8; 5] # Coût
cout_Materiaux = [9;5]
cout_Temps = [1;1]
Materiaux = 45
Temps_dispo = 6

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Produit[1:Nb_Var] >= 0, Int)

# define objective function
@objective(model, Max, sum(C.*Produit))

# define constraints
@constraint(model, sum(cout_Materiaux.*Produit) <= Materiaux)
@constraint(model, sum(cout_Temps.*Produit) <= Temps_dispo)

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 41.25 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (2 integer (0 of which binary)) and 4 elements
Cbc0012I Integer solution of -34 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0012I Integer solution of -40 found by DiveCoefficient after 1 iterations and 0 nodes (0.00 seconds)
Cbc0031I 1 added rows had average density of 2
Cbc0013I At root node, 1 cuts changed objective from -41.25 to -40 in 2 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 2 column cuts (2 active)  in 0.000 seconds - new frequency is 1
Cbc0014I Cut generator 1 (Gomory) - 1 row cuts average 2.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is 1
Cbc0014I Cut generator 2 (Knapsack) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds 

#### Tests avec l’instance disponible sur le sujet

In [24]:
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:Nb_Var
    println("\t Quantite de Produit $i = $(value(Produit[i]))")
end

la solution est : 
	 coût = 40.0
	 Quantite de Produit 1 = 5.0
	 Quantite de Produit 2 = 0.0


### 3.5 En bourse
***

La société de construction DELTA a réalisé d’importants bénéfices au cours de 3 dernières
années et envisage de faire fructifier une partie de ses gains en bourse. Pour sa première tenta-
tive, elle est prête à investir à hauteur de 5 millions d’e. Les marchés étant particulièrement
volatiles en ce moment, le conseiller financier suggère de ne pas mettre plus de 25% de la
somme investie dans un même produit financier et de ne pas dépasser un risque global de
2.0 ∗ . De plus, il serait intéressant d’investir au moins 30% dans les métaux précieux, qui
sont en général rentables et au moins 45% dans les crédits commerciaux et obligations, en
général fiables.
— Dans ces conditions, déduire quelle serait alors la stratégie d’investissement qui maxi-
mise les intérêts de DELTA ?
Produits financiers
Taux d’intérêt
Crédits commerciaux
7%
Obligations de sociétés 10%
Stocks d’Or
19%
Stocks de Platine
12%
Titres hypothécaires
8%
Prêts de construction
14%
coef de risque
1.7
1.2
3.7
2.4
2.0
2.9

#### Modélisation d'une forme générique

In [26]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
Nb_Var = 6 # Nombre de Variables
C = [0.07; 0.1; 0.19; 0.12; 0.08; 0.14] # Coût
Seuil_Max = [0.25;0.25;0.25;0.25;0.25;0.25]
Cout_Risque = [1.7; 1.2; 3.7; 2.4; 2.0; 2.9]
Risque = 2
Limitation = 5000000
Metaux_speciaux = 0.3
Investissement_fiable = 0.45

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, Variables[1:Nb_Var] >= 0)

# define objective function
@objective(model, Max, sum(C.*Variables))

# define constraints
@constraint(model, sum(Cout_Risque.*Variables) <= Risque*sum(Variables))
@constraint(model, Variables[3]+Variables[4] >= Metaux_speciaux*sum(Variables))
@constraint(model, Variables[1]+Variables[2] >= Investissement_fiable*sum(Variables))

for i in 1:Nb_Var
    @constraint(model, Variables[i] <= Seuil_Max[i]*sum(Variables))
end

@constraint(model, sum(Variables) <= Limitation)

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 10 (0) rows, 6 (0) columns and 59 (0) elements
0  Obj -0 Dual inf 0.6999994 (6)
0  Obj -0 Dual inf 0.6999994 (6)
11  Obj 520000
Optimal - objective value 520000
Optimal objective 520000 - 11 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



#### Tests avec l’instance disponible sur le sujet

In [27]:
println("la solution est : ")
println("\t coût = $(objective_value(model))")
for i in 1:Nb_Var
    println("\t Quantite de Variables $i = $(value(Variables[i]))")
end

la solution est : 
	 coût = 520000.0
	 Quantite de Variables 1 = 1.25e6
	 Quantite de Variables 2 = 1.25e6
	 Quantite de Variables 3 = 250000.0
	 Quantite de Variables 4 = 1.25e6
	 Quantite de Variables 5 = 500000.00000000006
	 Quantite de Variables 6 = 500000.0000000002
